<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part One: Data Input </h2>	


In [ ]:
# Mount Google Drive when working on Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd "/content/drive/My Drive/computer_vision"
    %pwd
    %ls
except:
    pass

Mounted at /content/drive
/content/drive/My Drive/computer_vision
 20_categories_training/    X_training_bovw_c150.pkl
 20_Validation/             X_training_bovw_c250.pkl
 bovw_vectorizer.pkl        X_training_bovw_c50.pkl
 cifar/                     X_training_bovw_c64.pkl
 figures/                   X_training_bovw_c791.pkl
 GradProject_NB1.ipynb      X_training_combined.pkl
 GradProject_NB2.ipynb      X_training.pkl
 GradProject_NB3.ipynb      X_train_tensor.pt
 GradProject_NB4.ipynb      X_train_tensor_standardized.pt
 imagenet20/                X_unlabeled.pkl
 model_utils.py             Y_nn_test.pkl
 prediction.csv             Y_nn_train.pkl
'Project Narrative.ipynb'   Y_testing_bovw_c150.pkl
 __pycache__/               Y_testing_bovw_c250.pkl
 results/                   Y_testing_bovw_c50.pkl
 testing_data.pkl           Y_testing_bovw_c64.pkl
 training_data.pkl          Y_testing_bovw_c791.pkl
 unlabeled_data.pkl         Y_testing_combined.pkl
 X_nn_test.pkl              Y_tes

<h3>  Import Statements </h3>	


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import skimage
from skimage import data
from skimage import io
import re
import os
from sklearn.model_selection import train_test_split

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<h2>Read Data</h2>

<h4> Take the given folder and create a dataframe with the picture object, and the encoding as listed below. </h4>	
0=Airplanes, 1=Bear, 2=Blimp, 3=Comet, 4=Crab, 5=Dog, 6=Dolphin, 7=Giraffe, 8=Goat, 9=Gorilla, 10=Kangaroo, 11=Killer-Whale, 12=Leopards, 13=Llama, 14= Penguin, 15= Porcupine, 16=Teddy-Bear, 17=Triceratops, 18=Unicorn, 19=Zebra

We found that the encoding is the alphanumeric order of labels, which can be obtained by sorting folders with their names.

In [ ]:
def read_organize_training_data(file_path):
    legal_category_pattern = r'[a-z]+-?[a-z]+'
    legal_img_pattern = r'[a-z]+-?[a-z]+_{1}\d+\.{1}jpg'

    pictures = []
    encodings = []
    label = 0
    n_grayscale_imgs = 0

    for category_name in sorted(os.listdir(file_path)):
        if re.fullmatch(legal_category_pattern, category_name) and os.path.isdir(os.path.join(file_path, category_name)):
            print("category name " + category_name + " " + "label to assign: " + str(label))
            category_path = os.path.join(file_path, category_name)
            for filename in os.listdir(category_path):
                if re.fullmatch(legal_img_pattern, filename) and os.path.isfile(os.path.join(category_path, filename)):
                    try:
                        filepath = os.path.join(category_path, filename)
                        img = io.imread(filepath)
                        if len(img.shape) != 3:
                            n_grayscale_imgs += 1
                            img = skimage.color.gray2rgb(img)
                        pictures.append(img)
                        encodings.append(label)
                    except:
                        print ("Error occured while reading: " + filepath)
            label += 1
    print(f'There are {len(pictures)-n_grayscale_imgs} RGB images, {n_grayscale_imgs} grayscale images.')
    return pd.DataFrame(data={'Pictures': pictures, 'Encoding': encodings})

In [ ]:
starting_data = read_organize_training_data("./20_categories_training/20_categories_training")
starting_data.shape

category name airplanes label to assign: 0
category name bear label to assign: 1
category name blimp label to assign: 2
category name comet label to assign: 3
category name crab label to assign: 4
category name dog label to assign: 5
category name dolphin label to assign: 6
category name giraffe label to assign: 7
category name goat label to assign: 8
category name gorilla label to assign: 9
category name kangaroo label to assign: 10
category name killer-whale label to assign: 11
category name leopards label to assign: 12
category name llama label to assign: 13
category name penguin label to assign: 14
category name porcupine label to assign: 15
category name teddy-bear label to assign: 16
category name triceratops label to assign: 17
category name unicorn label to assign: 18
category name zebra label to assign: 19
There are 1485 RGB images, 16 grayscale images.


(1501, 2)

In [ ]:
def read_organize_unlabeled_data(file_path):
    legal_img_pattern = r'.+.jpg'

    pictures = []
    filenames = []
    n_grayscale_imgs = 0

    for filename in os.listdir(file_path):
        if re.fullmatch(legal_img_pattern, filename) and os.path.isfile(os.path.join(file_path, filename)):
            try:
                filepath = os.path.join(file_path, filename)
                img = io.imread(filepath)
                if len(img.shape) != 3:
                            n_grayscale_imgs += 1
                            img = skimage.color.gray2rgb(img)
                pictures.append(img)
                filenames.append(filename)
            except:
                print ("Error occured while reading: " + filepath)
    print(f'There are {len(pictures)-n_grayscale_imgs} RGB images, {n_grayscale_imgs} grayscale images.')
    return pd.DataFrame(data={'Pictures': pictures, 'Filename': filenames})
unlabeled_data = read_organize_unlabeled_data("20_Validation")

There are 705 RGB images, 11 grayscale images.


In [ ]:
unlabeled_data['FileID'] = unlabeled_data['Filename'].str.extract(r'validation_pic \(([0-9]+)\).jpg', expand=False).astype(int)
unlabeled_data = unlabeled_data.sort_values('FileID').reset_index().drop(columns=['FileID','index'])
unlabeled_data.shape

(716, 2)

In [ ]:
unlabeled_data.head()

,Pictures,Filename
0,"[[[80, 79, 74], [56, 55, 50], [54, 53, 48], [4...",validation_pic (1).jpg
1,"[[[10, 15, 19], [10, 15, 19], [11, 16, 19], [1...",validation_pic (2).jpg
2,"[[[232, 231, 247], [250, 249, 255], [235, 234,...",validation_pic (3).jpg
3,"[[[97, 75, 77], [98, 76, 78], [99, 77, 79], [1...",validation_pic (4).jpg
4,"[[[19, 46, 55], [19, 46, 53], [20, 47, 54], [2...",validation_pic (5).jpg


<h2> Split and Save Data</h2>

Because the "20_Validation" dataset is unlabeled, we cannot evaluate the performance of our classifiers on it. So, we split the labeled dataset to a training set and a testing set.

In [ ]:
# 80/20 split for training/testing respectively
training_data, testing_data = train_test_split(starting_data, test_size=0.2)
training_data.shape, testing_data.shape

((1200, 2), (301, 2))

Store data in Pickle files. It saves memory compared to importing the whole notebook, and preserves class information compared to saving in CSV format.

In [ ]:
training_data.to_pickle('training_data.pkl')
testing_data.to_pickle('testing_data.pkl')
unlabeled_data.to_pickle('unlabeled_data.pkl')